In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,119 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,132 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [920 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,21

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_df=spark.read.csv(SparkFiles.get("home_sales_revised.csv"),sep=",",header=True)
home_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 2. Create a temporary view of the DataFrame.

home_df.createOrReplaceTempView('home_sales')

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

four_bed_avg=spark.sql(""" select round(avg(price),2), year(date) from home_sales where bedrooms = 4 group by year(date) order by year(date) """)
four_bed_avg.show()

+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|            300263.7|      2019|
|           298353.78|      2020|
|           301819.44|      2021|
|           296363.88|      2022|
+--------------------+----------+



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

three_bed_bath_avg=spark.sql(""" select round(avg(price),2), date_built from home_sales where bedrooms = 3 and bathrooms = 3 group by date_built order by date_built """)
three_bed_bath_avg.show()

+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           292859.62|      2010|
|           291117.47|      2011|
|           293683.19|      2012|
|           295962.27|      2013|
|           290852.27|      2014|
|            288770.3|      2015|
|           290555.07|      2016|
|           292676.79|      2017|
+--------------------+----------+



In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
three_bed_bath_floors_avg=spark.sql(""" select round(avg(price),2), date_built from home_sales where bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living >= 2000 group by date_built order by date_built """)

three_bed_bath_floors_avg.show()


+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           285010.22|      2010|
|           276553.81|      2011|
|           307539.97|      2012|
|           303676.79|      2013|
|           298264.72|      2014|
|           297609.97|      2015|
|            293965.1|      2016|
|           280317.58|      2017|
+--------------------+----------+



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql(""" select round(avg(price),2), view from home_sales group by view having avg(price) >= 350000 order by view""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           1026669.5| 100|
|           788128.21|  51|
|           733780.26|  52|
|            755214.8|  53|
|           798684.82|  54|
|           771153.32|  55|
|            718176.4|  56|
|            734340.5|  57|
|           759764.65|  58|
|            791453.0|  59|
|           754939.65|  60|
|           746877.59|  61|
|           759150.14|  62|
|           711614.55|  63|
|           767036.67|  64|
|           736679.93|  65|
|            712475.0|  66|
|           737970.96|  67|
|           716785.44|  68|
|           750537.94|  69|
+--------------------+----+
only showing top 20 rows

--- 1.9417171478271484 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
spark.sql(""" select round(avg(price),2), view from home_sales group by view having avg(price) <= 350000 order by view""").show()


print("--- %s seconds ---" % (time.time() - start_time))


+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           285069.21|   0|
|           288248.11|   1|
|           289693.49|  10|
|           280356.07|  11|
|           286575.22|  12|
|           285849.95|  13|
|           282519.32|  14|
|           284907.04|  15|
|           291990.83|  16|
|           286601.22|  17|
|           287532.36|  18|
|           280347.62|  19|
|           287710.06|   2|
|           284820.43|  20|
|           283451.36|  21|
|           284908.42|  22|
|           286977.37|  23|
|           281878.86|  24|
|           288583.21|  25|
|           281224.95|  26|
+--------------------+----+
only showing top 20 rows

--- 1.361219882965088 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_df.write.partitionBy("date_built").mode("overwrite").parquet("home_parquet")

In [ ]:
# 11. Read the parquet formatted data.
parquet_df=spark.read.parquet('home_parquet')

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('parquet_temp')

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql(""" select round(avg(price),2), view from parquet_temp group by view having avg(price) < 350000 order by view""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           285069.21|   0|
|           288248.11|   1|
|           289693.49|  10|
|           280356.07|  11|
|           286575.22|  12|
|           285849.95|  13|
|           282519.32|  14|
|           284907.04|  15|
|           291990.83|  16|
|           286601.22|  17|
|           287532.36|  18|
|           280347.62|  19|
|           287710.06|   2|
|           284820.43|  20|
|           283451.36|  21|
|           284908.42|  22|
|           286977.37|  23|
|           281878.86|  24|
|           288583.21|  25|
|           281224.95|  26|
+--------------------+----+
only showing top 20 rows

--- 0.6885335445404053 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [1]:
# 15. Check if the home_sales is no longer cached

if spark.catalog.isCached('home_sales') :
  print("a table is still cached")
else:
  print("all clear")

NameError: name 'spark' is not defined